# Hyperparameter tuning using Optuna 
resources:
1. https://aetperf.github.io/2021/02/16/Optuna-+-XGBoost-on-a-tabular-dataset.html
2.https://medium.com/optuna/using-optuna-to-optimize-xgboost-hyperparameters-63bfcdfd3407#:~:text=Optuna%20is%20a%20hyperparameter%20optimization,highly%20efficient%2C%20flexible%20and%20portable.
3.https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

In [60]:
# install optuna

!pip install optuna

In [61]:
# imports 

import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback, LightGBMPruningCallback
from sklearn.metrics import mean_squared_error
import holidays

In [62]:
# read data
train = pd.read_csv('./train_E1GspfA.csv', parse_dates=['date'])

In [63]:
# set datetime as index

train.set_index(pd.to_datetime(train.date) + pd.to_timedelta(train.hour, unit='h'), inplace=True)

In [64]:
# train-test split

# train
X_train = train.loc['2018':'2020'].drop(columns=['demand']).copy()
y_train = train.loc['2018':'2020', 'demand'].values.copy()

# test
X_test = train.loc['2021'].drop(columns=['demand']).copy()
y_test = train.loc['2021', 'demand'].values.copy()


# Feature engineering

In [65]:
# convert datetime to day, month, year

def convert_datetime_to_day_month_year(df):
    
    date_col = df.index
    
    df['day'] = date_col.day
    df['month'] = date_col.month
    df['year'] = date_col.year
    
    return df
    

In [66]:
# get day of week

def day_of_week(df):
    
    date_col = df.index
    
    df['day_of_week'] = date_col.weekday
    return df

In [67]:
# if weekend or weekday

def is_weekend(df):
    
    date_col = df.index
    
    df['is_weekend'] = np.where(df['day_of_week']<5, 0, 1)
    return df

In [68]:
# week of year

def week_of_year(df):
    
    date_col = df.index
    
    df['week_of_year'] = date_col.isocalendar().week.astype(int)
    
    return df

In [69]:
# Add holidays

def is_holiday(df):
    
    date_col = df.index.date
    
    Indian_holidays = holidays.India()
    US_holidays = holidays.US()
    
    df['is_holiday'] = [1 if (i in Indian_holidays) | (i in US_holidays) else 0 for i in date_col]
    return df


In [70]:
# whether it is day or night

def is_day(df):
    
    df['is_day'] = df.apply(lambda row: 1 if (row['hour']>=6 and row['hour']<=18) else 0, axis=1)
    
    return df
    

In [71]:
# seasons

def which_season(df):
    
    df['season'] = df.month%12 // 3 + 1
    
    return df

In [72]:
# convert hour to sin and cosine components

def cyclic_hour(df):
    
    hours_in_day = 24
    
    df['sin_hour'] = np.sin(2.*np.pi*df['hour']/hours_in_day)
    df['cos_hour'] = np.cos(2.*np.pi*df['hour']/hours_in_day)
    
    return df

In [73]:
# convert days of week into sin and cosine component

def cyclic_week(df):
    
    # monday = 0, sunday = 6
    days_in_week = 6
    
    df['sin_week'] = np.sin(2.*np.pi*df['day_of_week']/days_in_week)
    df['cos_week'] = np.cos(2.*np.pi*df['day_of_week']/days_in_week)
    
    return df

In [74]:
# convert months into sin and cosine components

def cyclic_month(df):
    
    months_in_year =12
    
    df['sin_month'] = np.sin(2.*np.pi*df['month']/months_in_year)
    df['cos_month'] = np.cos(2.*np.pi*df['month']/months_in_year)
    
    return df

In [75]:
# Add new features

def add_features(df):
    
    # add day, month, year
    df = convert_datetime_to_day_month_year(df)
    
    # day of week
    df = day_of_week(df)
    
    # is weekend
    df = is_weekend(df)
    
    # week of year
    df = week_of_year(df)
    
    # add holidays
    df = is_holiday(df)
    
    # is day
    df = is_day(df)
    
    # which season
    df = which_season(df)
    
    # # cyclic features
    df = cyclic_hour(df)
    df = cyclic_month(df)
    df = cyclic_week(df)

    return df

In [76]:
# add new features

X_train_features = add_features(X_train)
X_train_features.drop(columns=['date'], inplace=True)

X_test_features = add_features(X_test)
X_test_features.drop(columns=['date'], inplace=True)

In [77]:
# objective

def objective (trial, model, X_train, y_train, X_test, y_test,
               random_state = 42, n_jobs = -1, early_stopping_rounds =100):

  # XGBoost parameters
  xgb_params = {
      'tree_method': 'gpu_hist',
      'verbosity': 0,
      'objective':'reg:squarederror',
      'n_estimators': 10000,
      'max_depth': trial.suggest_int('max_depth', 4, 12),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
      'subsample': trial.suggest_loguniform('subsample', 0.6, 0.8),
      'min_child_weight': trial.suggest_loguniform('min_child_weight', 10, 100)
      }
  
  # LightGBM parameters

  lgbm_params = {
      # "device_type": 'gpu',
      "n_estimators": trial.suggest_int("n_estimators", 500,1000, step=100),
      "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
      "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
      "max_depth": trial.suggest_int("max_depth", 3, 12),
      "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 1000, step=100),
      "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
      "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
      "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.8, step=0.1),
      "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
      "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.8, step=0.1)
      }


  if model == 'xgb':
  
    # xgb
    xgb_model = XGBRegressor(** xgb_params)

    # prune not promising trails based on eval_set rmse
    xgb_pruning_callback = XGBoostPruningCallback(trial, 'validation_0-rmse')

    xgb_model.fit(X_train, y_train, 
            eval_set=[(X_test, y_test)],
            eval_metric = 'rmse',
            callbacks = [xgb_pruning_callback],
            early_stopping_rounds= early_stopping_rounds,
            )
    
    y_pred = xgb_model.predict(X_test)

  elif model == 'lgbm':

    # lightGBM
    lgbm_model = LGBMRegressor(** lgbm_params)

    # prune not promising trails based on eval_set rmse
    lgbm_pruning_callback = LightGBMPruningCallback(trial, 'rmse')

    lgbm_model.fit(X_train, y_train,
                  eval_set =[(X_test, y_test)],
                  eval_metric = 'rmse',
                  callbacks = [lgbm_pruning_callback],
                  early_stopping_rounds = early_stopping_rounds,
                  )
    
    y_pred = lgbm_model.predict(X_test)

    
  else:

    print('Provide a valid model name')
  
  return mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
# sampler

sampler = TPESampler(seed = 42, multivariate=True)

# study

model = 'lgbm'
study = create_study(direction='minimize', sampler=sampler, study_name = f'{model}_tuning')
study.optimize(
    lambda trial:
    objective(trial, model, X_train_features, y_train, X_test_features, y_test),
    n_trials = 500
)

In [79]:
study.best_params

{'bagging_fraction': 0.30000000000000004,
 'bagging_freq': 1,
 'feature_fraction': 0.7,
 'lambda_l1': 65,
 'lambda_l2': 55,
 'learning_rate': 0.09639177677050871,
 'max_depth': 9,
 'min_child_weight': 71.89768582681245,
 'min_data_in_leaf': 100,
 'n_estimators': 1000,
 'num_leaves': 80,
 'subsample': 0.6854090170878597}

In [80]:
# display params

best_params = study.best_params
for key, value in best_params.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

           max_depth : 9
       learning_rate : 0.09639177677050871
           subsample : 0.6854090170878597
    min_child_weight : 71.89768582681245
        n_estimators : 1000
          num_leaves : 80
    min_data_in_leaf : 100
           lambda_l1 : 65
           lambda_l2 : 55
    bagging_fraction : 0.30000000000000004
        bagging_freq : 1
    feature_fraction : 0.7
best objective value : 32.7397257590521
